In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome() 

url='https://zu.fang.com/'
driver.get(url) # 让浏览器打开指定网页

In [3]:
# 点击昌平
driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div/dl[1]/dd/a[7]').click()  
# 点击沙河
driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div[1]/div/a[13]').click()  

In [5]:
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait

# 存储所有租房数据
all_rental_data = []

# 最大爬取页数
max_pages = 20
current_page = 1

print("开始爬取租房数据...")

# 开始循环爬取每一页
while current_page <= max_pages:
    print(f"正在爬取第 {current_page} 页...")
    
    # 等待房源列表加载完成
    wait = WebDriverWait(driver, 10)
    house_list = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'houseList')))
    
    # 找到当前页的所有房子（每个dl就是一个房子）
    houses = house_list.find_elements(By.TAG_NAME, 'dl')
    print(f"第 {current_page} 页找到 {len(houses)} 个房源")
    
    # 遍历当前页的每一个房子
    for i, house in enumerate(houses):
        try:
            # 提取房子的文本信息
            house_text = house.text
            
            # 将数据添加到列表
            all_rental_data.append([house_text])
            
        except Exception as e:
            print(f"  提取第 {i+1} 个房源时出错: {e}")
            continue
    
    # 翻页
    if current_page < max_pages:
        try:
            # 查找下一页按钮
            next_button = driver.find_element(By.LINK_TEXT, "下一页")
            
            # 检查是否已经是最后一页
            if "disable" in next_button.get_attribute("class"):
                print("已经是最后一页，爬取结束")
                break
            
            # 点击下一页
            print("点击下一页...")
            next_button.click()
            
            # 等待新页面加载
            time.sleep(3)
            
            # 页码增加
            current_page += 1
            
        except NoSuchElementException:
            print("找不到下一页按钮，爬取结束")
            break
        except Exception as e:
            print(f"翻页时出错: {e}")
            break
    else:
        print(f"已完成 {max_pages} 页爬取")
        break

print("爬取完成！")

开始爬取租房数据...
正在爬取第 1 页...
第 1 页找到 60 个房源
点击下一页...
正在爬取第 2 页...
第 2 页找到 60 个房源
点击下一页...
正在爬取第 3 页...
第 3 页找到 60 个房源
点击下一页...
正在爬取第 4 页...
第 4 页找到 60 个房源
点击下一页...
正在爬取第 5 页...
第 5 页找到 60 个房源
点击下一页...
正在爬取第 6 页...
第 6 页找到 60 个房源
点击下一页...
正在爬取第 7 页...
第 7 页找到 60 个房源
点击下一页...
正在爬取第 8 页...
第 8 页找到 60 个房源
点击下一页...
正在爬取第 9 页...
第 9 页找到 60 个房源
点击下一页...
正在爬取第 10 页...
第 10 页找到 60 个房源
点击下一页...
正在爬取第 11 页...
第 11 页找到 60 个房源
点击下一页...
正在爬取第 12 页...
第 12 页找到 60 个房源
点击下一页...
正在爬取第 13 页...
第 13 页找到 60 个房源
点击下一页...
正在爬取第 14 页...
第 14 页找到 60 个房源
点击下一页...
正在爬取第 15 页...
第 15 页找到 60 个房源
点击下一页...
正在爬取第 16 页...
第 16 页找到 60 个房源
点击下一页...
正在爬取第 17 页...
第 17 页找到 60 个房源
点击下一页...
正在爬取第 18 页...
第 18 页找到 60 个房源
点击下一页...
正在爬取第 19 页...
第 19 页找到 60 个房源
点击下一页...
正在爬取第 20 页...
第 20 页找到 60 个房源
已完成 20 页爬取
爬取完成！


In [6]:
# 提取有用的数据
extracted_data = []
import re

print("开始处理提取数据...")

for i, rental_info in enumerate(all_rental_data):
    try:
        text = rental_info[0]
        
        # 初始化字段
        property_name = ""
        total_area = ""
        monthly_rent = ""
        
        # 1. 名称：第一行的前五个字
        lines = text.split('\n')
        if lines:
            first_line = lines[0].strip()
            # 正则匹配前五个中文字符
            chinese_chars = re.findall(r'[\u4e00-\u9fa5]', first_line)
            if len(chinese_chars) >= 5:
                property_name = ''.join(chinese_chars[:5])
            else:
                property_name = first_line[:5] if len(first_line) >= 5 else first_line
        
        # 2. 提取房屋总面积：匹配数字+㎡
        area_match = re.search(r'(\d+\.?\d*)\s*㎡', text)
        if area_match:
            total_area = area_match.group(1)
        
        # 3. 提取月租金：正则匹配数字+元/月
        rent_match = re.search(r'(\d+\.?\d*)\s*元/月', text)
        if rent_match:
            monthly_rent = rent_match.group(1)
        
        extracted_data.append([property_name, total_area, monthly_rent])
        
    except Exception as e:
        print(f"处理第 {i+1} 条数据时出错: {e}")
        extracted_data.append(["", "", ""])
        continue
print("数据提取完成！")

# 创建DataFrame并保存
if extracted_data:
    df = pd.DataFrame(extracted_data, columns=["名称", "面积(㎡)", "月租金(元)"])
    
    # 数据清洗：移除完全空白的行
    df_clean = df[(df["名称"] != "") | (df["面积(㎡)"] != "") | (df["月租金(元)"] != "")]
    
    # 保存到CSV文件
    output_file = "RentShaHe.csv"
    df_clean.to_csv("RentShaHe.csv", index=False, encoding='utf_8_sig')
    
    print(f"\nFinished!")
    print(f"Original num: {len(all_rental_data)}")
    print(f"Useful num: {len(df_clean)}")
    
    # 预览
    print("\nPreview:")
    print(df_clean.head(10))
    
else:
    print("nothing")

开始处理提取数据...
数据提取完成！

Finished!
Original num: 1200
Useful num: 1200

Preview:
      名称 面积(㎡) 月租金(元)
0  碧水庄园优   570  50000
1  正南北京玫   475  50000
2  北京玫瑰园   233  22000
3  碧水庄园区   375  23000
4  北京玫瑰园   271  17000
5  兆丰家园醇    86   3800
6  北京昌平兆    60   3600
7  兆丰家园朝    74   1300
8  北街家园八    35   2500
9  巩华家园东    86   4000
